In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
# Define parameters
concA = 1
concB = 2
k1 = 3.0
k2 = 0.6
k3 = 0.25
k4 = 2.95
vol = 30 #100.0

concA = 10
concB = 20
k1 = 6
k2 = 1.0
k3 = 230
k4 = 1000
vol = 8

In [ ]:
# Define CME birth/death rates
def lambdan(n):
    return concA*k1*n*(n-1)/vol + concB*k3*vol
def mun(n):
    return k2*n*(n-1)*(n-2)/vol**2 + n*k4

# Define intensity functions for tau-leaping
def lambda1(n):
    return concA*k1*n*(n-1)/vol
def lambda2(n):
    return k2*n*(n-1)*(n-2)/vol**2
def lambda3(n):
    return concB*k3*vol
def lambda4(n):
    return  n*k4

# Define terminal condition function
def terminalCondition(n):
    return 1.0*n

In [ ]:
# Do tau-leap integration loop
def target_func(X0):
    N = np.zeros([timesteps, len(X0)])
    for j in range(len(X0)):
        N[0,j] = X0[j]
    for i in range(timesteps-1):
        for j in range(len(X0)):
            numReactions1 = np.random.poisson(lambda1(N[i,j])*dt, 1)
            numReactions2 = np.random.poisson(lambda2(N[i,j])*dt, 1)
            numReactions3 = np.random.poisson(lambda3(N[i,j])*dt, 1)
            numReactions4 = np.random.poisson(lambda4(N[i,j])*dt, 1)
            N[i+1,j] = N[i,j] + numReactions1 - numReactions2 + numReactions3 - numReactions4
            if N[i+1,j] < 0:
                N[i+1,j] = 0
            
    return terminalCondition(N[-1,:])
# Note one advantage is to calculate several terminal conditions at once

In [ ]:
# Simulation parameters
timesteps = 10000
dt = 0.0001

# Network parameters
DOMAIN = [10, 200]   

# network settings
WIDTH = 10 #35 #20              # number of neurons per hidden layer
DEPTH = 8 #10 #6               # number of hidden layers
ACTIVATION = tf.nn.relu # hidden layer activation function

# training settings
INIT_L_RATE  = 1e-3 #2e-1
FINAL_L_RATE = 1e-5 #1e-3
NUM_ITER     = 4000 #8000
BATCH_SIZE   = 1024 #2048

# Other
OUTPUT_RES = 50
datasize = 10*BATCH_SIZE
FILENAME = "data/schlogl_data_v" + str(vol) + "_" + str(datasize) + ".txt"

In [ ]:
# Generate network structure

# Inputs and targets placeholders for trained data
networkInput = tf.placeholder(dtype=tf.float32, shape=(None,1), name='input')
networkTarget = tf.placeholder(dtype=tf.float32, shape=(None,1), name='target')

# Hidden layers
hidden = (DEPTH-1)*[None]
hidden[0] = networkInput
#next_layer = networkInput
for l in range(DEPTH-2):
    #hidden[l+1]= tf.layers.dense(next_layer, WIDTH, activation=ACTIVATION)
    #next_layer = tf.concat( hidden[:l+1], axis=1)
    
    #without skip connections
    hidden[l+1] = tf.layers.dense(hidden[l], WIDTH, activation=ACTIVATION)
    
# Add predition outermost layer
networkPrediction = tf.layers.dense(hidden[DEPTH-2], 1, activation=None, name='prediction')
# With skip connections
#networkPrediction = tf.layers.dense(tf.concat(hidden, axis=1), 1, activation=None, name='prediction')

In [ ]:
# Training procedure

# Define loss function and error
loss = tf.reduce_mean(tf.square(networkPrediction - networkTarget))
error = tf.reduce_max(tf.abs(networkPrediction - networkTarget))

# use stochastic gradient descent with ADAM during optimization
step = tf.train.AdamOptimizer(INIT_L_RATE).minimize(loss)

In [ ]:
# Print all parameters
print('\n----------------------------------------------------') 
print(' RUNNING EXPERIMENT WITH THE FOLLOWING PARAMETERS: ')
print('----------------------------------------------------\n')
print('depth:\t\t\t{}'.format(DEPTH))
print('width:\t\t\t{}'.format(WIDTH))
print('number of neurons:\t{}'.format(2+(DEPTH-2)*WIDTH))
print('number of connections:\t{}'.format(1+(DEPTH-2)*WIDTH*2+WIDTH*WIDTH*(DEPTH-3)*(DEPTH-2)//2))
print('activation:\t\t{}'.format(ACTIVATION.__name__))
print('learning rate:\t\t{} to {}'.format(INIT_L_RATE, FINAL_L_RATE))
print('iterations:\t\t{}'.format(NUM_ITER))
print('batch size:\t\t{}'.format(BATCH_SIZE))
print('\n\n')

In [ ]:
# Generate Input/target data in parallel
generatedata = False

# Do tau-leap integration loop (only for sclar initial condition)
def target_func_one(X0):
    N = np.zeros(timesteps)
    N[0] = X0
    for i in range(timesteps-1):
        numReactions1 = np.random.poisson(lambda1(N[i])*dt, 1)
        numReactions2 = np.random.poisson(lambda2(N[i])*dt, 1)
        numReactions3 = np.random.poisson(lambda3(N[i])*dt, 1)
        numReactions4 = np.random.poisson(lambda4(N[i])*dt, 1)
        N[i+1] = N[i] + numReactions1 - numReactions2 + numReactions3 - numReactions4
        if N[i+1] < 0:
            N[i+1] = 0
    return N


if generatedata:
    stride = 10
    x0 = np.float32(np.random.randint( DOMAIN[0], DOMAIN[1], datasize ))
    def propagate(x):
        xt = np.float32(target_func_one(x))
        y0 = np.zeros(int(timesteps/stride))
        for j in range(int(timesteps/stride)):
            y0[j] = 1.0*xt[j*stride]
        return y0
    
    num_cores = multiprocessing.cpu_count() 
    results = Parallel(n_jobs=num_cores, verbose = 2)(delayed(propagate)(i) for i in x0)
    print("Writing to file ...", end="\r")
    f = open(FILENAME,"w")
    for i in range(len(results)):
        f.write(" ".join(str(x) for x in results[i]) + "\n")
    f.close()
    print("Percentage finished:", 100, "%    ", end="\r")
    
# OLD NON-PARALLEL VERSION
# if generatedata:
#     timesteps = 10000
#     dt = 0.1
#     datasize = 50000
#     stride = 10
#     f = open(FILENAME,"w")
#     for i in range(datasize):
#         x0 = np.float32(np.random.randint( DOMAIN[0], DOMAIN[1] ))
#         y0 = np.float32(target_func_one(x0))
#         #f.write(str(x0) + ' ')
#         for j in range(int(timesAteps/stride)):
#             f.write(str(y0[j*stride]) + ' ')
#         f.write("\n")
#         #f.write(str(x0) + ' ' + str(y0) + "\n")
#         if i%100 == 0:
#             print("Percentage finished:", 100.0*float(i)/datasize, "%    ", end="\r")
#     f.close()
#     print("Percentage finished:", 100, "%    ", end="\r")

# else:
#     timesteps = 1000
#     dt = 0.1
#     datasize = 8000
#     inputData = (np.reshape(np.random.randint( DOMAIN[0], DOMAIN[1], datasize ) , [-1, 1])).astype('float32')
#     targetData = (np.reshape( target_func(inputData), [-1, 1])).astype('float32')

In [ ]:
readGenerateddata = True
if readGenerateddata:
    data = np.genfromtxt(FILENAME, delimiter=' ')
    inputData = data[:,0]
    targetData = terminalCondition(data[:,-1])   

In [ ]:
targetData = terminalCondition(data[:,-1])  

In [ ]:
# Start Tensorflow session and initialize all network variables
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
# Run the training

# run gradient descent steps with Adam
print('\nStarted training...')
print('{:8s}\t{:8s}\t{:8s}'.format('iter', 'l2-loss', 'linf-err'))
print('{:8s}\t{:8s}\t{:8s}'.format(*(3*[8*'-'])))
for iter in range(NUM_ITER):
    # generate random batch of inputs and corresponding target values
    #indices = (np.linspace(0,BATCH_SIZE-1,BATCH_SIZE)).astype(int)
    indices =  np.reshape(np.random.randint(0, datasize, BATCH_SIZE), [-1,1])
    inputBatch = np.take( inputData, indices)
    targetBatch = np.take( targetData, indices)

    # take gradient descent step and compute loss & error
    loss_val, error_val, _ = session.run(
        [loss, error, step],
        feed_dict={networkInput: inputBatch, networkTarget: targetBatch}
    )
    if iter % 100 == 0:
        print('{:8d}\t{:1.2e}\t{:1.2e}'.format(iter, loss_val, error_val))
print('...finished training.\n')

In [ ]:
# Plot for scalar case
plt.rcParams['figure.figsize'] = (12,9)
# generate full sample grid of input domain
RESOLUTION = 200
xgrid = np.linspace(DOMAIN[0], DOMAIN[1], num=RESOLUTION)
xgrid = xgrid.astype(int)
input_test_batch = np.reshape(xgrid , [-1,1])

# get model predictions
prediction_test_batch = session.run( networkPrediction, feed_dict={networkInput: input_test_batch})

# get actual target values and compare with predictions
#target_test_batch = target_func(xgrid, ygrid)
#l2_err = 1/2*np.mean(np.square(prediction_test_batch-target_test_batch))
#linf_err = np.max(np.abs(prediction_test_batch-target_test_batch))
#print(
#    'Error of predictions after training, evaluated on {}x{} grid:'
#    .format(RESOLUTION, RESOLUTION)
#)
#print('l2:\t{:1.4e}'.format(l2_err))
#print('l2inf:\t{:1.4e}'.format(linf_err))

# plot actual target, prediction, and comparison
plt.plot(inputData, targetData, '.', label = "Data")
plt.plot(input_test_batch, prediction_test_batch, '-r', linewidth=5, label = "NN")
plt.xlim([5,200])
plt.ylim([-20,200])
plt.xlabel('Initial value $X[0]$', fontsize = 20)
plt.ylabel('$X[T]$', fontsize = 20)
#plt.title('target function')
plt.legend(fontsize = 20)

plt.show()

## TESTS

In [ ]:
numReactions1 = np.random.poisson(lambda1(20)*dt, 1)
numReactions2 = np.random.poisson(lambda2(20)*dt, 1)
numReactions3 = np.random.poisson(lambda3(20)*dt, 1)
numReactions4 = np.random.poisson(lambda4(20)*dt, 1)

In [ ]:
print(numReactions1, numReactions2, numReactions3, numReactions4)